# IMPORT

In [ ]:
import pandas as pd
import numpy as np
import  requests
import io
from matplotlib import pyplot as plt
import seaborn as sns

# LECTURA

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_porcentajes = pd.read_csv('/content/gdrive/MyDrive/melchine_learning/henry_telecomunicaciones/porcentaaje.csv')

In [ ]:
df_porcentajes

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs,Banda ancha fija,Dial up
0,2022,4,BUENOS AIRES,78.94,36,5,6.0
1,2022,4,CAPITAL FEDERAL,122.73,36,2,2.0
2,2022,4,CATAMARCA,65.77,36,71,1.0
3,2022,4,CHACO,43.62,36,144,5.0
4,2022,4,CHUBUT,85.57,36,168,904.0
...,...,...,...,...,...,...,...
859,2014,1,SANTA CRUZ,28.94,1,26,460.0
860,2014,1,SANTA FE,47.85,1,506,612.0
861,2014,1,SANTIAGO DEL ESTERO,16.31,1,37,9.0
862,2014,1,TIERRA DEL FUEGO,63.97,1,27,766.0


# CREACION DEL KPI

In [ ]:
df_crecimiento = df_porcentajes.copy()
df_crecimiento = df_crecimiento.drop(["Banda ancha fija","Dial up"],axis=1)
df_crecimiento

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs
0,2022,4,BUENOS AIRES,78.94,36
1,2022,4,CAPITAL FEDERAL,122.73,36
2,2022,4,CATAMARCA,65.77,36
3,2022,4,CHACO,43.62,36
4,2022,4,CHUBUT,85.57,36
...,...,...,...,...,...
859,2014,1,SANTA CRUZ,28.94,1
860,2014,1,SANTA FE,47.85,1
861,2014,1,SANTIAGO DEL ESTERO,16.31,1
862,2014,1,TIERRA DEL FUEGO,63.97,1


In [ ]:
df_crecimiento["kpi"] = df_crecimiento.sort_values("TrimestreAbs").groupby("Provincia")["accesos_%home_prov"].transform("diff")
df_crecimiento["kpi"] = df_crecimiento["kpi"].replace(np.nan,0)
df_crecimiento["kpi"] = df_crecimiento["kpi"] / df_crecimiento["accesos_%home_prov"].apply(lambda v:max(v,1))

df_crecimiento

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs,kpi
0,2022,4,BUENOS AIRES,78.94,36,0.010514
1,2022,4,CAPITAL FEDERAL,122.73,36,0.003667
2,2022,4,CATAMARCA,65.77,36,0.006690
3,2022,4,CHACO,43.62,36,-0.005502
4,2022,4,CHUBUT,85.57,36,0.013907
...,...,...,...,...,...,...
859,2014,1,SANTA CRUZ,28.94,1,0.000000
860,2014,1,SANTA FE,47.85,1,0.000000
861,2014,1,SANTIAGO DEL ESTERO,16.31,1,0.000000
862,2014,1,TIERRA DEL FUEGO,63.97,1,0.000000


In [ ]:
df_crecimiento[df_crecimiento["Provincia"]=="BUENOS AIRES"].head(10)

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs,kpi
0,2022,4,BUENOS AIRES,78.94,36,0.010514
24,2022,3,BUENOS AIRES,78.11,35,0.006785
48,2022,2,BUENOS AIRES,77.58,34,0.019335
72,2022,1,BUENOS AIRES,76.08,33,0.005521
96,2021,4,BUENOS AIRES,75.66,32,0.052604
120,2021,3,BUENOS AIRES,71.68,31,0.023438
144,2021,2,BUENOS AIRES,70.00,30,0.012714
168,2021,1,BUENOS AIRES,69.11,29,0.001881
192,2020,4,BUENOS AIRES,68.98,28,0.019136
216,2020,3,BUENOS AIRES,67.66,27,0.036063


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Supongamos que ya tienes cargados tus datos en un DataFrame llamado df_crecimiento

# Ordena el DataFrame por provincia y trimestre para asegurarte de que los datos estén en el orden correcto
df_crecimiento.sort_values(by=['Provincia', 'TrimestreAbs'], inplace=True)

# Calcula el crecimiento acumulado por trimestre para cada provincia
df_crecimiento['Crecimiento_Acumulado'] = 100 * df_crecimiento['kpi']

# Obtén la lista de provincias únicas y años únicos para los widgets
provincias = df_crecimiento['Provincia'].unique()
anios = df_crecimiento['Año'].unique()

def update_grafico_todos(columna_trimestre,titulo,anio_seleccionado):
    plt.figure(figsize=(12, 6))
    data = df_crecimiento[(df_crecimiento['Año'].isin(anio_seleccionado))]
    for provincia, provincia_data in data.groupby('Provincia'):
        plt.plot(provincia_data[columna_trimestre], provincia_data['Crecimiento_Acumulado'], label=provincia)
    plt.xlabel('Trimestre')
    plt.ylabel('Crecimiento (%)')
    plt.title(titulo)
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1), borderaxespad=0.0)
    plt.show()

# Función para actualizar el gráfico en función de la provincia y el año seleccionados
def update_grafico(provincia_seleccionada, anio_seleccionado):
    if anio_seleccionado=="Todos":
      columna_trimestre = 'TrimestreAbs'
      titulo = f'Crecimiento en hogares para {provincia_seleccionada}'
      anio_seleccionado = anios
    else:
        columna_trimestre = 'Trimestre'
        titulo = f'Crecimiento en hogares para {provincia_seleccionada} en el año {anio_seleccionado}'
        anio_seleccionado = [anio_seleccionado]

    if provincia_seleccionada=="Todos":
      return update_grafico_todos(columna_trimestre,titulo,anio_seleccionado)

    plt.figure(figsize=(12, 6))

    data = df_crecimiento[(df_crecimiento['Provincia'] == provincia_seleccionada) & (df_crecimiento['Año'].isin(anio_seleccionado))]
    plt.plot(data[columna_trimestre], data['Crecimiento_Acumulado'])
    plt.xlabel('Trimestre')
    plt.ylabel('Crecimiento (%)')
    plt.title(titulo)
    plt.show()

# Crea los widgets de desplegable y conecta las funciones de actualización
provincia_dropdown = widgets.Dropdown(
    options=["Todos"]+provincias.tolist(),
    description='Provincia:',
)

anio_dropdown = widgets.Dropdown(
    options=["Todos"]+anios.tolist(),
    description='Año:',
)

widgets.interactive(update_grafico, provincia_seleccionada=provincia_dropdown, anio_seleccionado=anio_dropdown)


interactive(children=(Dropdown(description='Provincia:', options=('Todos', 'BUENOS AIRES', 'CAPITAL FEDERAL', …

In [ ]:
df_crecimiento.to_csv("/content/gdrive/MyDrive/melchine_learning/henry_telecomunicaciones/crecimiento_hogares.csv",index=False)

# KPI PROPUESTO

> Queremos llevar la tecnologia wireless a las provincias que les falte, la idea es poder tener un incremento anual del 10%

In [183]:
df_tecnologias_prov = pd.read_csv('/content/gdrive/MyDrive/melchine_learning/henry_telecomunicaciones/accesos_tecnologia_provincia.csv')
df_tecnologias_prov

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,4,Buenos Aires,326.0,3.0,1.0,130.0,72.0,5.0
1,2022,4,Capital Federal,139.0,1.0,139.0,5.0,32.0,2.0
2,2022,4,Catamarca,9.0,11.0,47.0,1.0,2.0,71.0
3,2022,4,Chaco,25.0,61.0,47.0,8.0,3.0,144.0
4,2022,4,Chubut,45.0,74.0,10.0,31.0,9.0,169.0
...,...,...,...,...,...,...,...,...,...
860,2014,1,Santa Fe,323.0,174.0,3.0,6.0,644.0,507.0
861,2014,1,Santiago Del Estero,33.0,4.0,19.0,915.0,34.0,37.0
862,2014,1,Tierra Del Fuego,22.0,3.0,648.0,1.0,3.0,28.0
863,2014,1,Tucumán,130.0,83.0,121.0,13.0,98.0,130.0


In [184]:
df_crecimiento_wireless = df_tecnologias_prov.copy()
df_crecimiento_wireless = df_crecimiento_wireless.drop(["ADSL","Cablemodem","Fibra óptica","Otros","Total"],axis=1)
df_crecimiento_wireless["Provincia"] = df_crecimiento_wireless["Provincia"].str.upper()
df_crecimiento_wireless = df_crecimiento_wireless.dropna()
df_crecimiento_wireless

,Año,Trimestre,Provincia,Wireless
0,2022,4,BUENOS AIRES,130.0
1,2022,4,CAPITAL FEDERAL,5.0
2,2022,4,CATAMARCA,1.0
3,2022,4,CHACO,8.0
4,2022,4,CHUBUT,31.0
...,...,...,...,...
859,2014,1,SANTA CRUZ,4.0
860,2014,1,SANTA FE,6.0
861,2014,1,SANTIAGO DEL ESTERO,915.0
862,2014,1,TIERRA DEL FUEGO,1.0


In [185]:
df_crecimiento_wireless["Año"].unique().tolist()

['2022',
 '2021',
 '2020',
 '2019',
 '2019 *',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014']

In [186]:
df_crecimiento_wireless["Año"] = df_crecimiento_wireless["Año"].str.strip()
df_crecimiento_wireless["Año"] = df_crecimiento_wireless["Año"].str.replace("*","")
df_crecimiento_wireless["Año"] = pd.to_numeric(df_crecimiento_wireless["Año"])

df_crecimiento_wireless["Trimestre"] = df_crecimiento_wireless["Trimestre"].str.strip()
df_crecimiento_wireless["Trimestre"] = df_crecimiento_wireless["Trimestre"].str.replace("*","")
df_crecimiento_wireless["Trimestre"] = pd.to_numeric(df_crecimiento_wireless["Trimestre"])

df_crecimiento_wireless

<ipython-input-186-51d64fec2c4d>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_crecimiento_wireless["Año"] = df_crecimiento_wireless["Año"].str.replace("*","")
<ipython-input-186-51d64fec2c4d>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_crecimiento_wireless["Trimestre"] = df_crecimiento_wireless["Trimestre"].str.replace("*","")


,Año,Trimestre,Provincia,Wireless
0,2022,4,BUENOS AIRES,130.0
1,2022,4,CAPITAL FEDERAL,5.0
2,2022,4,CATAMARCA,1.0
3,2022,4,CHACO,8.0
4,2022,4,CHUBUT,31.0
...,...,...,...,...
859,2014,1,SANTA CRUZ,4.0
860,2014,1,SANTA FE,6.0
861,2014,1,SANTIAGO DEL ESTERO,915.0
862,2014,1,TIERRA DEL FUEGO,1.0


In [187]:
def get_trimestre_abs(trimestre,anio):
  return trimestre+4*(int(anio)-df_crecimiento_wireless["Año"].min())

df_crecimiento_wireless["TrimestreAbs"] = df_crecimiento_wireless.apply(lambda r:get_trimestre_abs(r["Trimestre"],r["Año"]),axis=1)
df_crecimiento_wireless

,Año,Trimestre,Provincia,Wireless,TrimestreAbs
0,2022,4,BUENOS AIRES,130.0,36
1,2022,4,CAPITAL FEDERAL,5.0,36
2,2022,4,CATAMARCA,1.0,36
3,2022,4,CHACO,8.0,36
4,2022,4,CHUBUT,31.0,36
...,...,...,...,...,...
859,2014,1,SANTA CRUZ,4.0,1
860,2014,1,SANTA FE,6.0,1
861,2014,1,SANTIAGO DEL ESTERO,915.0,1
862,2014,1,TIERRA DEL FUEGO,1.0,1


In [188]:
df_crecimiento_wireless["kpi"] = df_crecimiento_wireless.sort_values("TrimestreAbs").groupby("Provincia")["Wireless"].transform("diff")
df_crecimiento_wireless["kpi"] = df_crecimiento_wireless["kpi"].replace(np.nan,0)
df_crecimiento_wireless["kpi"] = df_crecimiento_wireless["kpi"] / df_crecimiento_wireless["Wireless"].apply(lambda v:max(v,1))

df_crecimiento_wireless

,Año,Trimestre,Provincia,Wireless,TrimestreAbs,kpi
0,2022,4,BUENOS AIRES,130.0,36,0.023077
1,2022,4,CAPITAL FEDERAL,5.0,36,-0.200000
2,2022,4,CATAMARCA,1.0,36,0.000000
3,2022,4,CHACO,8.0,36,0.000000
4,2022,4,CHUBUT,31.0,36,0.032258
...,...,...,...,...,...,...
859,2014,1,SANTA CRUZ,4.0,1,0.000000
860,2014,1,SANTA FE,6.0,1,0.000000
861,2014,1,SANTIAGO DEL ESTERO,915.0,1,0.000000
862,2014,1,TIERRA DEL FUEGO,1.0,1,0.000000


In [189]:

import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Supongamos que ya tienes cargados tus datos en un DataFrame llamado df_crecimiento_wireless

# Ordena el DataFrame por provincia y trimestre para asegurarte de que los datos estén en el orden correcto
df_crecimiento_wireless.sort_values(by=['Provincia', 'TrimestreAbs'], inplace=True)

# Calcula el crecimiento acumulado por trimestre para cada provincia
df_crecimiento_wireless['Crecimiento_Acumulado'] = 100 * df_crecimiento_wireless['kpi']

# Obtén la lista de provincias únicas y años únicos para los widgets
provincias = df_crecimiento_wireless['Provincia'].unique()
anios = df_crecimiento_wireless['Año'].unique()

def update_grafico_todos(columna_trimestre,titulo,anio_seleccionado):
    plt.figure(figsize=(12, 6))
    data = df_crecimiento_wireless[(df_crecimiento_wireless['Año'].isin(anio_seleccionado))]
    for provincia, provincia_data in data.groupby('Provincia'):
        plt.plot(provincia_data[columna_trimestre], provincia_data['Crecimiento_Acumulado'], label=provincia)
    plt.xlabel('Trimestre')
    plt.ylabel('Crecimiento (%)')
    plt.title(titulo)
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1), borderaxespad=0.0)
    plt.show()

# Función para actualizar el gráfico en función de la provincia y el año seleccionados
def update_grafico(provincia_seleccionada, anio_seleccionado):
    if anio_seleccionado=="Todos":
      columna_trimestre = 'TrimestreAbs'
      titulo = f'Crecimiento de Wireless para {provincia_seleccionada}'
      anio_seleccionado = anios
    else:
        columna_trimestre = 'Trimestre'
        titulo = f'Crecimiento de Wireless para {provincia_seleccionada} en el año {anio_seleccionado}'
        anio_seleccionado = [anio_seleccionado]

    if provincia_seleccionada=="Todos":
      return update_grafico_todos(columna_trimestre,titulo,anio_seleccionado)

    plt.figure(figsize=(12, 6))

    data = df_crecimiento_wireless[(df_crecimiento_wireless['Provincia'] == provincia_seleccionada) & (df_crecimiento_wireless['Año'].isin(anio_seleccionado))]
    plt.plot(data[columna_trimestre], data['Crecimiento_Acumulado'])
    plt.xlabel('Trimestre')
    plt.ylabel('Crecimiento (%)')
    plt.title(titulo)
    plt.show()

# Crea los widgets de desplegable y conecta las funciones de actualización
provincia_dropdown = widgets.Dropdown(
    options=["Todos"]+provincias.tolist(),
    description='Provincia:',
)

anio_dropdown = widgets.Dropdown(
    options=["Todos"]+anios.tolist(),
    description='Año:',
)

widgets.interactive(update_grafico, provincia_seleccionada=provincia_dropdown, anio_seleccionado=anio_dropdown)


interactive(children=(Dropdown(description='Provincia:', options=('Todos', 'BUENOS AIRES', 'CAPITAL FEDERAL', …

In [ ]:
df_crecimiento_wireless.to_csv("/content/gdrive/MyDrive/melchine_learning/henry_telecomunicaciones/crecimiento_wireless.csv",index=False)